In [1]:
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
import torch
import torch.utils.data as data_utils

In [2]:
model_checkpoint = "distilbert-base-uncased"
num_labels = 9
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
file = "Data/clean_unstemmed.csv"

In [3]:
from datasets import load_dataset, load_metric
dataset = load_dataset('csv', data_files=file,split = 'train')

Using custom data configuration default-a2933d159e599270
Reusing dataset csv (/Users/michaelbutler/.cache/huggingface/datasets/csv/default-a2933d159e599270/0.0.0/2a88c45fed596f9421a2e7f74ab1a3cd012ef75210a5dc1950e8d60ca8d9c66c)


In [4]:
dataset.features

{'idx': Value(dtype='int64', id=None),
 'headline': Value(dtype='string', id=None),
 'summary': Value(dtype='string', id=None),
 'labels': Value(dtype='int64', id=None)}

In [5]:
dataset = dataset.train_test_split()

In [6]:
def preprocess(entry):
    return tokenizer(entry["headline"],entry["summary"],truncation=True)
encoded_dataset = dataset.map(preprocess,batched= True)

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [7]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [8]:
batch_size = 16

args = TrainingArguments(
    "test-nytimes",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

In [9]:
from sklearn.metrics import accuracy_score,f1_score

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    
    results = accuracy_score(predictions,labels, normalize = True )#metric.compute(predictions=predictions, references=labels)
    return {
        "accuracy": results 
    }

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [10]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,No log,1.300055,0.539434,210.397000,10.908000
2,1.439300,1.250829,0.543791,425.837300,5.389000
3,1.173600,1.268368,0.551634,177.631900,12.920000
4,0.940800,1.321624,0.541176,250.220700,9.172000
5,0.780900,1.376890,0.532462,180.347300,12.725000


TrainOutput(global_step=2155, training_loss=1.0571531999415422, metrics={'train_runtime': 12726.737, 'train_samples_per_second': 0.169, 'total_flos': 794703219905394.0, 'epoch': 5.0})